Importing libraries

In [70]:
# Importing the necessary libraries
import pandas as pd
import matplotlib.pyplot as plt

Read data to dataframe

In [84]:
#Creating the dataframe
sales_data= pd.read_csv("all_data.csv")

#Getting an overview of the data
print("columns= ",sales_data.columns,"\nno of rows and columns= ",sales_data.shape)

Cleaning up the data

In [72]:
#Viewing the empty rows
nan_rows= sales_data[sales_data.isnull().any(axis=1)]

#Drop the empty rows
sales_data= sales_data.dropna(how="all")

#Reset the index as the index would be changed as we dropped some rows
sales_data=sales_data.reset_index(drop=True)

#Converting columns to their correct data type
sales_data["Quantity Ordered"]= pd.to_numeric(sales_data["Quantity Ordered"], errors="coerce")
sales_data["Price Each"]=pd.to_numeric(sales_data["Price Each"], errors="coerce")


Question1: What was the best month for sales? 

Add a month column to the dataframe

In [73]:
sales_data["Month"]= sales_data["Order Date"].str[0:2]

#Convert the month column to integer so further analysis can be done with it
#sales_data["Month"]= sales_data["Month"].astype("int")
#at first when this code was tried it gave the error "ValueError: invalid literal for int() with base 10: 'Or'" because some of the months were appearing as "Or" instead of the correct month for some reason.

sales_data=sales_data[sales_data["Order ID"].str[0:2] != "Or"] #This is a condition for the rows with the "Or" to be ingnored and select the ones without it
sales_data["Month"]=sales_data["Month"].astype("int32")

Adding a Sales column

In [75]:
sales_data["Sales"]= sales_data["Quantity Ordered"] * sales_data["Price Each"]

#Rearranging the columns to place sales closer to the beginning
sales_data= sales_data.iloc[:,[0,1,2,3,7,4,5,6]]

Answering the question

In [86]:
result= sales_data.groupby("Month")[["Price Each", "Quantity Ordered","Sales"]].sum().reset_index()

#Plotting the result
plt.bar(result["Month"], result["Sales"])
plt.xticks(result["Month"])
plt.xlabel("Months")
plt.ylabel("Sales in USD")
plt.show()

Question2: What City has the highest number of Sales

Creating and extracting a city value

In [77]:
def get_city(address):
    return address.split(",")[1]

def get_code(address):
    return address.split(",")[2].split()[0]

sales_data["City"]= sales_data["Purchase Address"].apply(lambda x: (f"{get_city(x)}  {get_code(x)}"))

Answering the question

In [87]:
max_sale=sales_data.groupby("City")[["Sales"]].sum().reset_index()
plt.bar(max_sale["City"], max_sale["Sales"])
plt.xticks(max_sale["City"],rotation="vertical")
plt.xlabel("City Name")
plt.ylabel("Sales in USD")

plt.show()

What time should we post advertistements to maximize likelyhood of customers buying products

In [81]:
#Convert the order date column to date time format
sales_data["Order Date"]= pd.to_datetime(sales_data["Order Date"])

In [82]:
#Add hour,minutes column
sales_data["Hour"]=sales_data["Order Date"].dt.hour
sales_data["Minute"]= sales_data["Order Date"].dt.minute

In [88]:
hours=sales_data.groupby("Hour").count().reset_index()
plt.plot(hours["Hour"], hours)
plt.xticks(hours["Hour"])
plt.xlabel("Hours")
plt.ylabel("Hours Count")
plt.grid()
plt.show

What items are most often sold together 

In [89]:
#How to know the products that were sold together is that they would have the same order ID
sold_together=sales_data[sales_data["Order ID"].duplicated(keep=False)].reset_index()

#Create a new column that has the two products that were bought together and separate them with a comma
sold_together["Grouped"]= sold_together.groupby("Order ID")["Product"].transform(lambda x: ",". join (x))
grouped_products= sold_together[["Order ID", "Grouped"]].drop_duplicates()
grouped_products.head()

In [90]:
#Count the pairs that occurred the most
from itertools import combinations
from collections import Counter

count= Counter()
for row in grouped_products["Grouped"]:
    row_list= row.split(",")
    count.update(Counter(combinations(row_list, 2)))
for key, value in count.most_common(10):
    print(key, value)

What Product sold the most and why do you think it sold the most?

In [91]:
products_sold= sales_data.groupby("Product")["Quantity Ordered"].sum().reset_index()
quantity_ordered= products_sold["Quantity Ordered"]
plt.bar(products_sold["Product"], quantity_ordered)
plt.xticks(rotation="vertical")
plt.ylabel("Quantity Ordered")
plt.xlabel("Product")
plt.show()

Checking the correlation between the quantity ordered and the prices

In [92]:
prices= sales_data.groupby("Product")["Price Each"].mean()

fig, ax1= plt.subplots()
ax2=ax1.twinx()
ax1.bar(products_sold["Product"],quantity_ordered, color= "g")
ax2.plot(products_sold["Product"], prices, "b-")

ax1.set_xlabel("Product Name")
ax2.set_ylabel("Quantity Ordered", color="g")
ax1.set_xticklabels(products_sold["Product"],rotation="vertical")
plt.show()